# The simplest example - an elastic cube

**Notebook prepared by 

Joakim Sundnes**

Date: **Jun 25, 2018**


## Model outline
We start with a simple example, where we consider loading of a unit cube. This simple case will illustrate how to define a non-linear elasticity problem in Fenics, and how to introduce boundary conditions and different material properties. The model will be a simple unit cube, fixed at one end ($x=0$) and loaded with a pressure load (negative pressure, i.e. stretch) at the other end ($x=1.0$). The first version of the model will use a simple isotropic neo-Hookean material model. Later we will introduce a more realistic soft tissue model, and also add active contraction. 


### Weak form of the static hyper-elasticity problem
We want to solve a static solid mechanics problem with a combination of Dirichlet and Neumann boundary conditions:
$$
 \begin{alignat*}{2}
    - \nabla\cdot P &= 0 && \quad \text{ in } \Omega \\
    u &= 0 && \quad \text{ on } \Gamma_{\mathrm{D}} \\
    P \cdot n &= T && \quad \text{ on } \Gamma_{\mathrm{N}} \\
    P \cdot n &= 0 && \quad \text{ on } \Gamma_{\mathrm{0}}
  \end{alignat*}
$$
Here, $P$ is the first Piola-Kirchhoff stress tensor, $u$ is the displacement, $T$ is a load (vector) applied to parts of the boundary, and $\Omega, \Gamma_{\mathrm{D}}, \Gamma_{\mathrm{N}},\Gamma_{\mathrm{0}}$ are the domain and the boundaries for Dirichlet- and Neumann boundary conditions, respectively. 

To apply the finite element method, we need to derive the weak form of the problem.  Multiply by a test function $v \in \hat{V}$ and integrate by parts:
$$
  \begin{equation*}
    - \int_{\Omega} \nabla\cdot P \cdot v dx
    = \int_{\Omega} P : \nabla v dx - \int_{\partial\Omega} (P \cdot n) \cdot v ds = 0
  \end{equation*}
$$

We now apply the boundary conditions (and note that $v = 0$ on $\Gamma_{\mathrm{D}}$), to get the final weak form:

Find $u \in V$ such that
$$
\begin{equation*}
    \int_{\Omega} P : \nabla v dx
    = \int_{\Gamma_{\mathrm{N}}} T \cdot v ds
  \end{equation*}
$$
for all $v \in \hat{V}$ 

In our case the boundary load $T$ is a simple normal pressure. If we were working in the deformed configuration, we would simply have $T=-p n$, where $p$ is the applied pressure and $n$ is the unit surface normal. However, we are using a Lagrangian formulation and everything is defined relative to the reference state. The applied pressure, which actually acts on the deformed surface, therefore needs to be mapped to the reference state. We get
$$
\begin{equation*}
T = -p J F^{-T}\cdot N,
\end{equation*}
$$
where $N$ is the unit normal of the undeformed (reference) geometry. 

### The St Venant-Kirchhoff material model
The simplest hyper-elastic material model is the St Venant-Kirchhoff model, which is simply an extension of the linear Hooke's law to the large-deformation regime. The strain energy function is given by:
$$
\begin{equation*}
    W(E) = \frac{\lambda}{2} (tr(E))^2 + \mu tr(E^2),
  \end{equation*}
  $$
where  $E = \frac{1}{2}(F^T F - I)$ is the Green--Lagrange strain, and $F = I + \nabla u$ is the deformation gradient. From the strain energy function we can obtain the first and second Piola-Kirchhoff (PK) stress ($P$ and $S$) as
$$
\begin{align*}
P_{ij} &= \frac{\partial W}{\partial F_{ij}}, \\
S_{ij} &= \frac{\partial W}{\partial E_{ij}},
\end{align*}
$$
and we also have the relation $P = FS$. Both the first and second PK stresses are commonly used in finite element solvers. The formulation based on $S$ is popular in text books, as it allows a few more analytical steps in the derivation of the nonlinear solution method. However, in a computational setting, and in particular when using an automated system like FEniCS, the formulation based on $P$ is just as simple.

### The Fenics solver
We are now ready to specify the problem in Fenics. First, the usual imports and defining the solution domain, the function space and the test- and trial functions:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from fenics import *

# Optimization options for the form compiler
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["representation"] = "uflacs"
parameters["form_compiler"]["quadrature_degree"] = 4


# Setup the mesh and the function space for the solutions
mesh = UnitCubeMesh(4,4,4)
V = VectorFunctionSpace(mesh, "Lagrange", 2)


# Define functions
v  = TestFunction(V)             # Test function
u  = Function(V)                 # Displacement from previous iteration

Now we need to define the boundary conditions. We want homogenous Dirichlet conditions on the left boundary $x=0.0$, a non-homoegenous Neumann conditions on $x=1.0$, and homogenous Neumann conditions everywhere else. The following code first defines the subdomains and marks the respective boundaries, then redefines the boundary measure (ds) to allow surface integrals over parts of the boundary, and then defines the Dirichlet conditions:

In [ ]:
# Mark boundary subdomains
left =  CompiledSubDomain("near(x[0], side) && on_boundary", side = 0.0)
right = CompiledSubDomain("near(x[0], side) && on_boundary", side = 1.0)

boundary_markers = MeshFunction("size_t", mesh,mesh.topology().dim() - 1)
boundary_markers.set_all(0)
left.mark(boundary_markers, 1)
right.mark(boundary_markers, 2)

# Redefine boundary measure
ds = Measure('ds',domain=mesh,subdomain_data=boundary_markers)

# Define Dirichlet boundary (x = 0 or x = 1)
clamp = Constant((0.0, 0.0, 0.0))
bc = DirichletBC(V, clamp, left)
bcs = [bc]

Next, we turn to defining the mechanics problem. We start with the kinematics and the strain energy function defining the neo-Hookean material, obtain the Piola-Kirchhoff stresses by differentiating the strain energy function, and finally define the weak form of the problem.

In [ ]:
# Kinematics
d = len(u)
I = Identity(d)             # Identity tensor
F = I + grad(u)             # Deformation gradient
C = F.T*F                   # the right Cauchy-Green tensor
E = 0.5*(C - I)             # the Green-Lagrange strain tensor

# Material parameters (Lamé parameters)
mu    = 4.0  
lmbda = 20.0  

# The strain energy for the St-Venant Kirchhoff model:
E = variable(E)
psi = lmbda/2*(tr(E)**2) + mu*tr(E*E)

S = diff(psi,E) #second Piola-Kirchhoff stress
P = F*S         #First Piola-Kirchhoff stress

p_right = Constant(0.0) #the pressure load (zero for now)

# Definition of the weak form:
N = FacetNormal(mesh)
Gext = p_right * inner(v, cofac(F)*N) * ds(2) #ds(2) = left boundary
R = inner(P,grad(v))*dx + Gext 

Finally, we solve the problem for different loads, and plot the load vs displacement. 

In [ ]:


# The middle point on the right boundary
point0 = np.array([1.0,0.5,0.5])


# Step-wise loading (for plotting and convergence)
load_steps = 6
target_load = 10.0
loads = np.linspace(0,target_load,load_steps)

d0 = np.zeros(3)                #displacement at point0
disp = np.zeros(load_steps) #array to store displacement for all steps

for step in range(load_steps):
    # Stretch is a negative pressure
    p_right.assign(-loads[step])
    
    #solve the problem:
    solve(R == 0, u, bcs)
    
    #evaluate displacement at point defined above
    u.eval(d0,point0)
    disp[step] = d0[0]

    
#displacement on x-axis, load on y-axis
plt.figure(1)
plt.plot(disp,loads)
plt.xlabel('Displacement of point (1.0,0.5,0.5)')
plt.ylabel('Applied pressure load')

# Plot the deformed cube and the original
plt.figure(2)
u_int = dolfin.interpolate(u, dolfin.VectorFunctionSpace(mesh, "CG", 1))
deformed_mesh = dolfin.Mesh(mesh)
dolfin.ALE.move(deformed_mesh, u_int)
dolfin.plot(mesh, alpha=0.5, edgecolor='k')#, title="original")
dolfin.plot(deformed_mesh, edgecolor='g', alpha=0.7)#, title='Contracting cube')
ax = plt.gca()
ax.view_init(elev=2, azim=-92)

plt.show()

### Questions and comments:
* The neo-Hookean model is a linear stress-strain relation, but the curve above is non-linear. Why?
* An open source cardiac mechanics solver, based on the approach outlined above, can be found here: [https://github.com/finsberg/pulse](https://github.com/finsberg/pulse) 